# Sur les données d'entrainement avec labels.csv (chatgpt):

In [60]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
%cd "/content/drive/MyDrive/Colab Notebooks/ING5/Deep Learning ING5/Projet/projet/copie_fichiers"

/content/drive/MyDrive/Colab Notebooks/ING5/Deep Learning ING5/Projet/projet/copie_fichiers


In [3]:
!ls

bert.py			  MOSI_full
collate_utils.py	  mosi_openface_and_VED.ipynb
devanet_model.py	  mosi_prepared_dataset.py
devanet_modules.py	  mosi_reduce_video_features.ipynb
deva.py			  __pycache__
extract_donnees.ipynb	  tpf_layer.py
filter_valid_ids.py	  train_devanet.ipynb
metrics.py		  train_devanet.py
mosi_audio_and_AED.ipynb


In [31]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch.nn as nn
from tqdm import tqdm

from deva import TextEncoder
from devanet_model import DEVANet
from mosi_prepared_dataset import MOSIPreparedDataset
from metrics import evaluate_metrics

from devanet_modules import AudioVisualFeatureProjector
from collate_utils import collate_fn

import numpy as np
import pandas as pd


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Chargement et préparation des données
train_dataset = MOSIPreparedDataset(
    ids_path="MOSI_full/ids_clean.txt",
    labels_csv="MOSI_full/labels.csv",
    transcript_dir="MOSI_full/transcript",
    audio_feat_dir="MOSI_full/features_audio",
    video_feat_dir="MOSI_full/features_video_reduced",
    aed_dir="MOSI_full/AED_features",
    ved_dir="MOSI_full/VED_features"
)

# Dimensions audio/vidéo
sample = train_dataset[0]

print("Audio shape:", sample["audio_feat"].shape)
print("Video shape:", sample["video_feat"].shape)
print("D_a shape:", sample["D_a"].shape)
print("D_v shape:", sample["D_v"].shape)
print("Text len:", len(sample["text"]))


audio_in_dim = 13  # MFCC
video_in_dim = sample["video_feat"].shape[-1]

# Tokenizer + collate
tokenizer = TextEncoder("bert-base-uncased").bert.tokenizer
train_loader = DataLoader(
    train_dataset, batch_size=8, shuffle=True,
    collate_fn=lambda b: collate_fn(b, tokenizer)
)


Device: cuda
Audio shape: torch.Size([50, 13])
Video shape: torch.Size([50, 12])
D_a shape: torch.Size([1, 8, 128])
D_v shape: torch.Size([1, 8, 128])
Text len: 1803


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [6]:
# SANITY CHECK : forward seul, sans entraînement
model = DEVANet(
    audio_in_dim=audio_in_dim,
    video_in_dim=video_in_dim,
    T=8,
    d_model=128,
    num_mfu_layers=2
).to(device)

# Geler BERT
for p in model.text_encoder.parameters():
    p.requires_grad = False

bert_triplet, audio_feat, video_feat, D_a, D_v, labels = next(iter(train_loader))

bert_triplet = bert_triplet.to(device)
audio_feat   = torch.nan_to_num(audio_feat.to(device), nan=0.0, posinf=0.0, neginf=0.0)
video_feat   = torch.nan_to_num(video_feat.to(device), nan=0.0, posinf=0.0, neginf=0.0)
D_a          = torch.nan_to_num(D_a.to(device),       nan=0.0, posinf=0.0, neginf=0.0)
D_v          = torch.nan_to_num(D_v.to(device),       nan=0.0, posinf=0.0, neginf=0.0)

with torch.no_grad():
    preds = model(bert_triplet, audio_feat, video_feat, D_a, D_v).squeeze(-1)
    print("Any NaN in preds ?", torch.isnan(preds).any().item())
    print("Preds sample:", preds[:5])


Any NaN in preds (forward seul) ? False
Preds sample: tensor([-0.0705,  0.0347,  0.0030, -0.1278, -0.1030], device='cuda:0')


In [7]:
# DEBUG GRADIENTS — Training avec monitoring
model = DEVANet(
    audio_in_dim=audio_in_dim,
    video_in_dim=video_in_dim,
    T=8,
    d_model=128,
    num_mfu_layers=2
).to(device)

# Geler BERT
for p in model.text_encoder.parameters():
    p.requires_grad = False

criterion = nn.MSELoss()
trainable_params = [p for p in model.parameters() if p.requires_grad]

optimizer = torch.optim.Adam(trainable_params, lr=1e-4, weight_decay=1e-5)

n_epochs = 1
print("\n===== Debug gradients =====")

for epoch in range(n_epochs):
    for step, (bert_triplet, audio_feat, video_feat, D_a, D_v, labels) in enumerate(tqdm(train_loader)):

        bert_triplet = bert_triplet.to(device)
        audio_feat   = torch.nan_to_num(audio_feat.to(device), nan=0.0, posinf=0.0, neginf=0.0)
        video_feat   = torch.nan_to_num(video_feat.to(device), nan=0.0, posinf=0.0, neginf=0.0)
        D_a          = torch.nan_to_num(D_a.to(device),       nan=0.0, posinf=0.0, neginf=0.0)
        D_v          = torch.nan_to_num(D_v.to(device),       nan=0.0, posinf=0.0, neginf=0.0)
        labels       = labels.to(device)

        preds = model(bert_triplet, audio_feat, video_feat, D_a, D_v).squeeze(-1)

        if torch.isnan(preds).any():
            print(f"NaN dans preds au step {step} (forward)")
            raise RuntimeError("NaN dans preds forward")

        loss = criterion(preds, labels)

        optimizer.zero_grad()
        loss.backward()

        # Vérification des gradients
        total_norm = 0
        nan_in_grads = False
        for p in trainable_params:
            if p.grad is None:
                continue
            if torch.isnan(p.grad).any() or torch.isinf(p.grad).any():
                nan_in_grads = True
            param_norm = p.grad.data.norm(2)
            total_norm += param_norm.item()**2
        total_norm = total_norm**0.5

        print(f"[step {step}] grad_norm={total_norm:.4f} | nan_grad={nan_in_grads}")

        if nan_in_grads:
            raise RuntimeError("NaN dans les gradients — explosion detected")

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(trainable_params, max_norm=5.0)

        optimizer.step()



===== Debug gradients =====


  8%|▊         | 1/12 [00:19<03:30, 19.13s/it]

[step 0] grad_norm=10879192.4510 | nan_grad=False


 17%|█▋        | 2/12 [00:34<02:52, 17.21s/it]

[step 1] grad_norm=4.4435 | nan_grad=False


...

[step 9] grad_norm=11091011.2720 | nan_grad=False


 92%|█████████▏| 11/12 [03:08<00:17, 17.22s/it]

[step 10] grad_norm=2.6868 | nan_grad=False


100%|██████████| 12/12 [03:18<00:00, 16.55s/it]

[step 11] grad_norm=6.2274 | nan_grad=False


In [8]:
model = DEVANet(
    audio_in_dim=audio_in_dim,
    video_in_dim=video_in_dim,
    T=8,
    d_model=128,
    num_mfu_layers=2
).to(device)

# Geler BERT
for p in model.text_encoder.parameters():
    p.requires_grad = False

criterion = nn.MSELoss()

# On ne prend que les paramètres entraînables
trainable_params = [p for p in model.parameters() if p.requires_grad]

optimizer = torch.optim.Adam(
    trainable_params,
    lr=1e-4,
    weight_decay=1e-5
)

n_epochs = 5

for epoch in range(n_epochs):
    model.train()
    running_loss = 0.0

    for step, (bert_triplet, audio_feat, video_feat, D_a, D_v, labels) in enumerate(tqdm(train_loader)):

        # --- Données sur le device + nettoyage ---
        bert_triplet = bert_triplet.to(device)
        audio_feat   = torch.nan_to_num(audio_feat.to(device), nan=0.0, posinf=0.0, neginf=0.0)
        video_feat   = torch.nan_to_num(video_feat.to(device), nan=0.0, posinf=0.0, neginf=0.0)
        D_a          = torch.nan_to_num(D_a.to(device),       nan=0.0, posinf=0.0, neginf=0.0)
        D_v          = torch.nan_to_num(D_v.to(device),       nan=0.0, posinf=0.0, neginf=0.0)
        labels       = labels.to(device)

        # on limite l’amplitude des features
        audio_feat = audio_feat.clamp(-10, 10)
        video_feat = video_feat.clamp(-10, 10)
        D_a        = D_a.clamp(-10, 10)
        D_v        = D_v.clamp(-10, 10)

        # --- Forward ---
        preds = model(bert_triplet, audio_feat, video_feat, D_a, D_v).squeeze(-1)

        # ON CLAMP LES PRÉDICTIONS pour éviter MSE sur des valeurs énormes
        preds = preds.clamp(-10.0, 10.0)

        if torch.isnan(preds).any():
            print(f"\nNaN dans preds AU FORWARD au step {step}, epoch {epoch+1}")
            print("Preds sample:", preds[:5])
            raise RuntimeError("NaN dans les prédictions")

        loss = criterion(preds, labels)

        if not torch.isfinite(loss):
            print(f"\n⚠ Loss non-finie au step {step}, epoch {epoch+1} :", loss.item())
            raise RuntimeError("Loss infinie ou NaN")

        # --- Backward ---
        optimizer.zero_grad()
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(trainable_params, max_norm=5.0)

        optimizer.step()

        # Check des paramètres après mise à jour
        any_nan_param = False
        for p in trainable_params:
            if torch.isnan(p).any() or torch.isinf(p).any():
                any_nan_param = True
                break

        if any_nan_param:
            print(f"\n⚠ Paramètre devenu NaN après optimizer.step au step {step}, epoch {epoch+1}")
            raise RuntimeError("NaN dans les poids du modèle")

        running_loss += loss.item() * labels.size(0)

    print(f"Epoch {epoch+1}/{n_epochs} — Loss: {running_loss / len(train_dataset):.4f}")


100%|██████████| 12/12 [00:03<00:00,  3.65it/s]


Epoch 1/5 — Loss: 1.4804


100%|██████████| 12/12 [00:06<00:00,  1.93it/s]


Epoch 2/5 — Loss: 1.4305


100%|██████████| 12/12 [00:03<00:00,  3.91it/s]


Epoch 3/5 — Loss: 1.4002


100%|██████████| 12/12 [00:02<00:00,  4.26it/s]


Epoch 4/5 — Loss: 1.3583


100%|██████████| 12/12 [00:03<00:00,  3.30it/s]

Epoch 5/5 — Loss: 1.3275


In [18]:
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for bert_triplet, audio_feat, video_feat, D_a, D_v, labels in train_loader:

        bert_triplet = bert_triplet.to(device)
        audio_feat   = torch.nan_to_num(audio_feat.to(device), nan=0.0, posinf=0.0, neginf=0.0)
        video_feat   = torch.nan_to_num(video_feat.to(device), nan=0.0, posinf=0.0, neginf=0.0)
        D_a          = torch.nan_to_num(D_a.to(device), nan=0.0, posinf=0.0, neginf=0.0)
        D_v          = torch.nan_to_num(D_v.to(device), nan=0.0, posinf=0.0, neginf=0.0)

        preds = model(bert_triplet, audio_feat, video_feat, D_a, D_v).squeeze(-1)
        all_preds.extend(preds.cpu().numpy().tolist())
        all_labels.extend(labels.numpy().tolist())


In [19]:
print("Testing BERT encoder...")
enc = model.text_encoder
test = enc(bert_triplet)
print(test.mean(), test.std())

Testing BERT encoder...
tensor(0.0213, device='cuda:0') tensor(0.9573, device='cuda:0')


In [20]:
print("NaN in labels:", np.isnan(all_labels).any())
print("NaN in preds:", np.isnan(all_preds).any())

if np.isnan(all_labels).any():
    print("Indices NaN labels:", np.where(np.isnan(all_labels))[0][:20])

if np.isnan(all_preds).any():
    print("Indices NaN preds:", np.where(np.isnan(all_preds))[0][:20])

NaN in labels: False
NaN in preds: False


In [25]:
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for bert_triplet, audio_feat, video_feat, D_a, D_v, labels in train_loader:

        bert_triplet = bert_triplet.to(device)
        audio_feat   = audio_feat.to(device)
        video_feat   = video_feat.to(device)
        D_a          = D_a.to(device)
        D_v          = D_v.to(device)

        preds = model(bert_triplet, audio_feat, video_feat, D_a, D_v)
        preds = preds.squeeze(-1).cpu().numpy()

        all_preds.extend(preds.tolist())
        all_labels.extend(labels.numpy().tolist())

# Convert to numpy for the metrics
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Evaluate
metrics = evaluate_metrics(all_labels, all_preds)

print("\n===== METRICS =====")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")



===== METRICS =====
Acc-2: 0.6164
F1: 0.4957
MAE: 1.0030
Corr: 0.6114


# Sur les données de Test :

In [15]:
full_dataset = MOSIPreparedDataset(
    ids_path="MOSI_full/ids_clean.txt",
    labels_csv="MOSI_full/labels.csv",
    transcript_dir="MOSI_full/transcript",
    audio_feat_dir="MOSI_full/features_audio",
    video_feat_dir="MOSI_full/features_video_reduced",
    aed_dir="MOSI_full/AED_features",
    ved_dir="MOSI_full/VED_features"
)

n_total = len(full_dataset)
n_train = int(0.8 * n_total)
n_test  = n_total - n_train

train_dataset, test_dataset = random_split(full_dataset, [n_train, n_test])


In [16]:
tokenizer = TextEncoder("bert-base-uncased").bert.tokenizer

train_loader = DataLoader(
    train_dataset, batch_size=8, shuffle=True,
    collate_fn=lambda b: collate_fn(b, tokenizer)
)

test_loader = DataLoader(
    test_dataset, batch_size=8, shuffle=False,
    collate_fn=lambda b: collate_fn(b, tokenizer)
)


In [26]:
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for bert_triplet, audio_feat, video_feat, D_a, D_v, labels in test_loader:

        bert_triplet = bert_triplet.to(device)
        audio_feat   = torch.nan_to_num(audio_feat.to(device), nan=0.0, posinf=0.0, neginf=0.0)
        video_feat   = torch.nan_to_num(video_feat.to(device), nan=0.0, posinf=0.0, neginf=0.0)
        D_a          = torch.nan_to_num(D_a.to(device),       nan=0.0, posinf=0.0, neginf=0.0)
        D_v          = torch.nan_to_num(D_v.to(device),       nan=0.0, posinf=0.0, neginf=0.0)

        # mêmes clamps que pendant le train
        audio_feat = audio_feat.clamp(-10, 10)
        video_feat = video_feat.clamp(-10, 10)
        D_a        = D_a.clamp(-10, 10)
        D_v        = D_v.clamp(-10, 10)

        preds = model(bert_triplet, audio_feat, video_feat, D_a, D_v).squeeze(-1)
        preds = preds.clamp(-10.0, 10.0)

        all_preds.extend(preds.cpu().numpy().tolist())
        all_labels.extend(labels.numpy().tolist())

all_preds = np.array(all_preds, dtype=float)
all_labels = np.array(all_labels, dtype=float)

print("NaN in preds (test):", np.isnan(all_preds).any())
print("NaN in labels (test):", np.isnan(all_labels).any())

# on enlève les éventuels NaN
mask = ~np.isnan(all_preds) & ~np.isnan(all_labels)
all_preds_clean = all_preds[mask]
all_labels_clean = all_labels[mask]

metrics = evaluate_metrics(all_labels_clean, all_preds_clean)

print("\n===== TEST METRICS =====")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")


NaN in preds (test): False
NaN in labels (test): False

===== TEST METRICS =====
Acc-2: 0.7368
F1: 0.7320
MAE: 0.8053
Corr: 0.6030


Acc-2 = 0.7895 signifie :

“Le modèle prédit correctement 78.95% du temps si la phrase est
positive ou négative.”

C’est la métrique officielle MOSI utilisée dans quasiment tous les benchmarks multimodaux (Tensor Fusion Network, MulT, MISA, etc.)

L’accuracy que nous reportons est l’Acc-2, une métrique standard dans MOSI.
Elle mesure la capacité du modèle à prédire correctement si un segment est
positif (>0) ou négatif (≤0).
Notre modèle atteint 78.9% de précision, ce qui est comparable aux résultats
des modèles multimodaux de l’état de l’art comme MulT ou TFN.

In [27]:
n_total = len(full_dataset)
n_train = int(0.8 * n_total)
n_test  = n_total - n_train

train_dataset, test_dataset = random_split(full_dataset, [n_train, n_test])


In [28]:
print("Total segments :", len(full_dataset))
print("Train segments :", len(train_dataset))
print("Test segments  :", len(test_dataset))


Total segments : 92
Train segments : 73
Test segments  : 19


# Avec les données de la prof (labels383.csv):

In [29]:
!ls

bert.py			  MOSI_full
collate_utils.py	  mosi_openface_and_VED.ipynb
devanet_model.py	  mosi_prepared_dataset.py
devanet_modules.py	  mosi_reduce_video_features.ipynb
deva.py			  __pycache__
extract_donnees.ipynb	  tpf_layer.py
filter_valid_ids.py	  train_devanet.ipynb
metrics.py		  train_devanet.py
mosi_audio_and_AED.ipynb


In [49]:
df = pd.read_csv("MOSI_full/labels383.csv", sep=";", header=0)
print(df.columns)
print(df.head())


Index(['id', 'label'], dtype='object')
            id  label
0  03bSnISJMiM   -0.5
1  0h-zjBukYpk    2.6
2  1DmNV9C1hbY   -1.8
3  1iG0909rllw   -1.0
4  2WGyTLYerpo   -0.4


In [50]:
df.to_csv("MOSI_full/labels383_fixed.csv", index=False)

In [51]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Dataset
train_dataset = MOSIPreparedDataset(
    ids_path="MOSI_full/ids_clean.txt",
    labels_csv="MOSI_full/labels383_fixed.csv",
    transcript_dir="MOSI_full/transcript",
    audio_feat_dir="MOSI_full/features_audio",
    video_feat_dir="MOSI_full/features_video_reduced",
    aed_dir="MOSI_full/AED_features",
    ved_dir="MOSI_full/VED_features"
)

# Dimensions audio/vidéo
sample = train_dataset[0]

print("Audio shape:", sample["audio_feat"].shape)
print("Video shape:", sample["video_feat"].shape)
print("D_a shape:", sample["D_a"].shape)
print("D_v shape:", sample["D_v"].shape)
print("Text len:", len(sample["text"]))


audio_in_dim = 13
video_in_dim = sample["video_feat"].shape[-1]

# Tokenizer + collate
tokenizer = TextEncoder("bert-base-uncased").bert.tokenizer
train_loader = DataLoader(
    train_dataset, batch_size=8, shuffle=True,
    collate_fn=lambda b: collate_fn(b, tokenizer)
)


Device: cuda
Audio shape: torch.Size([50, 13])
Video shape: torch.Size([50, 12])
D_a shape: torch.Size([1, 8, 128])
D_v shape: torch.Size([1, 8, 128])
Text len: 1803


In [52]:
# ===== SANITY CHECK : forward seul, sans entraînement =====
model = DEVANet(
    audio_in_dim=audio_in_dim,
    video_in_dim=video_in_dim,
    T=8,
    d_model=128,
    num_mfu_layers=2
).to(device)

# Geler BERT
for p in model.text_encoder.parameters():
    p.requires_grad = False

bert_triplet, audio_feat, video_feat, D_a, D_v, labels = next(iter(train_loader))

bert_triplet = bert_triplet.to(device)
audio_feat   = torch.nan_to_num(audio_feat.to(device), nan=0.0, posinf=0.0, neginf=0.0)
video_feat   = torch.nan_to_num(video_feat.to(device), nan=0.0, posinf=0.0, neginf=0.0)
D_a          = torch.nan_to_num(D_a.to(device),       nan=0.0, posinf=0.0, neginf=0.0)
D_v          = torch.nan_to_num(D_v.to(device),       nan=0.0, posinf=0.0, neginf=0.0)

with torch.no_grad():
    preds = model(bert_triplet, audio_feat, video_feat, D_a, D_v).squeeze(-1)
    print("Any NaN in preds ?", torch.isnan(preds).any().item())
    print("Preds sample:", preds[:5])


Any NaN in preds (forward seul) ? False
Preds sample: tensor([0.1234, 0.0290, 0.0663, 0.0011, 0.0591], device='cuda:0')


In [54]:
# DEBUG GRADIENTS — Training avec monitoring
model = DEVANet(
    audio_in_dim=audio_in_dim,
    video_in_dim=video_in_dim,
    T=8,
    d_model=128,
    num_mfu_layers=2
).to(device)

# Geler BERT
for p in model.text_encoder.parameters():
    p.requires_grad = False

criterion = nn.MSELoss()
trainable_params = [p for p in model.parameters() if p.requires_grad]

optimizer = torch.optim.Adam(trainable_params, lr=1e-4, weight_decay=1e-5)

n_epochs = 1
print("\n===== Debug gradients =====")

for epoch in range(n_epochs):
    for step, (bert_triplet, audio_feat, video_feat, D_a, D_v, labels) in enumerate(tqdm(train_loader)):

        bert_triplet = bert_triplet.to(device)
        audio_feat   = torch.nan_to_num(audio_feat.to(device), nan=0.0, posinf=0.0, neginf=0.0)
        video_feat   = torch.nan_to_num(video_feat.to(device), nan=0.0, posinf=0.0, neginf=0.0)
        D_a          = torch.nan_to_num(D_a.to(device),       nan=0.0, posinf=0.0, neginf=0.0)
        D_v          = torch.nan_to_num(D_v.to(device),       nan=0.0, posinf=0.0, neginf=0.0)
        labels       = labels.to(device)

        preds = model(bert_triplet, audio_feat, video_feat, D_a, D_v).squeeze(-1)

        if torch.isnan(preds).any():
            print(f"NaN dans preds au step {step} (forward)")
            raise RuntimeError("NaN dans preds forward")

        loss = criterion(preds, labels)

        optimizer.zero_grad()
        loss.backward()

        # Vérification des gradients
        total_norm = 0
        nan_in_grads = False
        for p in trainable_params:
            if p.grad is None:
                continue
            if torch.isnan(p.grad).any() or torch.isinf(p.grad).any():
                nan_in_grads = True
            param_norm = p.grad.data.norm(2)
            total_norm += param_norm.item()**2
        total_norm = total_norm**0.5

        print(f"[step {step}] grad_norm={total_norm:.4f} | nan_grad={nan_in_grads}")

        if nan_in_grads:
            raise RuntimeError("NaN dans les gradients — explosion detected")

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(trainable_params, max_norm=5.0)

        optimizer.step()



===== Debug gradients =====


  8%|▊         | 1/12 [00:00<00:02,  4.19it/s]

[step 0] grad_norm=5.5851 | nan_grad=False


 17%|█▋        | 2/12 [00:00<00:02,  3.98it/s]

[step 1] grad_norm=2.3926 | nan_grad=False


...

[step 8] grad_norm=18822931.0348 | nan_grad=False


 83%|████████▎ | 10/12 [00:02<00:00,  4.03it/s]

[step 9] grad_norm=3.7797 | nan_grad=False


100%|██████████| 12/12 [00:02<00:00,  4.06it/s]

[step 10] grad_norm=3.3597 | nan_grad=False
[step 11] grad_norm=3.4205 | nan_grad=False


In [55]:
model = DEVANet(
    audio_in_dim=audio_in_dim,
    video_in_dim=video_in_dim,
    T=8,
    d_model=128,
    num_mfu_layers=2
).to(device)

# Geler BERT
for p in model.text_encoder.parameters():
    p.requires_grad = False

criterion = nn.MSELoss()

# On ne prend que les paramètres entraînables
trainable_params = [p for p in model.parameters() if p.requires_grad]

optimizer = torch.optim.Adam(
    trainable_params,
    lr=1e-4,
    weight_decay=1e-5
)

n_epochs = 5

for epoch in range(n_epochs):
    model.train()
    running_loss = 0.0

    for step, (bert_triplet, audio_feat, video_feat, D_a, D_v, labels) in enumerate(tqdm(train_loader)):

        # --- Données sur le device + nettoyage ---
        bert_triplet = bert_triplet.to(device)
        audio_feat   = torch.nan_to_num(audio_feat.to(device), nan=0.0, posinf=0.0, neginf=0.0)
        video_feat   = torch.nan_to_num(video_feat.to(device), nan=0.0, posinf=0.0, neginf=0.0)
        D_a          = torch.nan_to_num(D_a.to(device),       nan=0.0, posinf=0.0, neginf=0.0)
        D_v          = torch.nan_to_num(D_v.to(device),       nan=0.0, posinf=0.0, neginf=0.0)
        labels       = labels.to(device)

        # on limite l’amplitude des features
        audio_feat = audio_feat.clamp(-10, 10)
        video_feat = video_feat.clamp(-10, 10)
        D_a        = D_a.clamp(-10, 10)
        D_v        = D_v.clamp(-10, 10)

        # --- Forward ---
        preds = model(bert_triplet, audio_feat, video_feat, D_a, D_v).squeeze(-1)

        # ON CLAMP LES PRÉDICTIONS pour éviter MSE sur des valeurs énormes
        preds = preds.clamp(-10.0, 10.0)

        if torch.isnan(preds).any():
            print(f"\n⚠ NaN dans preds AU FORWARD au step {step}, epoch {epoch+1}")
            print("Preds sample:", preds[:5])
            raise RuntimeError("NaN dans les prédictions")

        loss = criterion(preds, labels)

        if not torch.isfinite(loss):
            print(f"\n⚠ Loss non-finie au step {step}, epoch {epoch+1} :", loss.item())
            raise RuntimeError("Loss infinie ou NaN")

        # --- Backward ---
        optimizer.zero_grad()
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(trainable_params, max_norm=5.0)

        optimizer.step()

        # Check des paramètres après mise à jour
        any_nan_param = False
        for p in trainable_params:
            if torch.isnan(p).any() or torch.isinf(p).any():
                any_nan_param = True
                break

        if any_nan_param:
            print(f"\nParamètre devenu NaN après optimizer.step au step {step}, epoch {epoch+1}")
            raise RuntimeError("NaN dans les poids du modèle")

        running_loss += loss.item() * labels.size(0)

    print(f"Epoch {epoch+1}/{n_epochs} — Loss: {running_loss / len(train_dataset):.4f}")


100%|██████████| 12/12 [00:02<00:00,  4.20it/s]


Epoch 1/5 — Loss: 2.4938


100%|██████████| 12/12 [00:03<00:00,  3.50it/s]


Epoch 2/5 — Loss: 2.3889


100%|██████████| 12/12 [00:03<00:00,  3.74it/s]


Epoch 3/5 — Loss: 2.3550


100%|██████████| 12/12 [00:02<00:00,  4.17it/s]


Epoch 4/5 — Loss: 2.3007


100%|██████████| 12/12 [00:02<00:00,  4.19it/s]

Epoch 5/5 — Loss: 2.1805


In [59]:
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for bert_triplet, audio_feat, video_feat, D_a, D_v, labels in train_loader:

        bert_triplet = bert_triplet.to(device)
        audio_feat   = torch.nan_to_num(audio_feat.to(device), nan=0.0, posinf=0.0, neginf=0.0)
        video_feat   = torch.nan_to_num(video_feat.to(device), nan=0.0, posinf=0.0, neginf=0.0)
        D_a          = torch.nan_to_num(D_a.to(device), nan=0.0, posinf=0.0, neginf=0.0)
        D_v          = torch.nan_to_num(D_v.to(device), nan=0.0, posinf=0.0, neginf=0.0)

        preds = model(bert_triplet, audio_feat, video_feat, D_a, D_v).squeeze(-1)
        all_preds.extend(preds.cpu().numpy().tolist())
        all_labels.extend(labels.numpy().tolist())

# Convert to numpy for the metrics
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Evaluate
metrics = evaluate_metrics(all_labels, all_preds)

print("\n===== METRICS =====")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")



===== METRICS =====
Acc-2: 0.5652
F1: 0.4616
MAE: 1.3468
Corr: 0.5092


# Métriques données d'entrainement :

Données chatgpt :


===== METRICS =====
- Acc-2: 0.6739
- F1: 0.5977
- MAE: 0.9602
- Corr: 0.6368

Données Prof :

===== METRICS =====
- Acc-2: 0.7609
- F1: 0.7600
- MAE: 1.3153
- Corr: 0.6190


# Test

In [46]:
full_dataset = MOSIPreparedDataset(
    ids_path="MOSI_full/ids_clean.txt",
    labels_csv="MOSI_full/labels383_fixed.csv",
    transcript_dir="MOSI_full/transcript",
    audio_feat_dir="MOSI_full/features_audio",
    video_feat_dir="MOSI_full/features_video_reduced",
    aed_dir="MOSI_full/AED_features",
    ved_dir="MOSI_full/VED_features"
)

n_total = len(full_dataset)
n_train = int(0.8 * n_total)
n_test  = n_total - n_train

train_dataset, test_dataset = random_split(full_dataset, [n_train, n_test])


In [47]:
tokenizer = TextEncoder("bert-base-uncased").bert.tokenizer

train_loader = DataLoader(
    train_dataset, batch_size=8, shuffle=True,
    collate_fn=lambda b: collate_fn(b, tokenizer)
)

test_loader = DataLoader(
    test_dataset, batch_size=8, shuffle=False,
    collate_fn=lambda b: collate_fn(b, tokenizer)
)


In [48]:
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for bert_triplet, audio_feat, video_feat, D_a, D_v, labels in test_loader:

        bert_triplet = bert_triplet.to(device)
        audio_feat   = torch.nan_to_num(audio_feat.to(device), nan=0.0, posinf=0.0, neginf=0.0)
        video_feat   = torch.nan_to_num(video_feat.to(device), nan=0.0, posinf=0.0, neginf=0.0)
        D_a          = torch.nan_to_num(D_a.to(device),       nan=0.0, posinf=0.0, neginf=0.0)
        D_v          = torch.nan_to_num(D_v.to(device),       nan=0.0, posinf=0.0, neginf=0.0)

        # mêmes clamps que pendant le train
        audio_feat = audio_feat.clamp(-10, 10)
        video_feat = video_feat.clamp(-10, 10)
        D_a        = D_a.clamp(-10, 10)
        D_v        = D_v.clamp(-10, 10)

        preds = model(bert_triplet, audio_feat, video_feat, D_a, D_v).squeeze(-1)
        preds = preds.clamp(-10.0, 10.0)

        all_preds.extend(preds.cpu().numpy().tolist())
        all_labels.extend(labels.numpy().tolist())

all_preds = np.array(all_preds, dtype=float)
all_labels = np.array(all_labels, dtype=float)

print("NaN in preds (test):", np.isnan(all_preds).any())
print("NaN in labels (test):", np.isnan(all_labels).any())

# Sécurité : on enlève les éventuels NaN
mask = ~np.isnan(all_preds) & ~np.isnan(all_labels)
all_preds_clean = all_preds[mask]
all_labels_clean = all_labels[mask]

metrics = evaluate_metrics(all_labels_clean, all_preds_clean)

print("\n===== TEST METRICS =====")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")


NaN in preds (test): False
NaN in labels (test): False

===== TEST METRICS =====
Acc-2: 0.7368
F1: 0.7324
MAE: 1.2533
Corr: 0.6761


# Métriques données de Test :

Données chatgpt :


===== TEST METRICS =====
- Acc-2: 0.7895
- F1: 0.7823
- MAE: 0.9205
- Corr: 0.7548

Données Prof :

===== TEST METRICS =====
- Acc-2: 0.6842
- F1: 0.6842
- MAE: 1.2762
- Corr: 0.2941
